##### Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import collections

##### Variaveis de teste

In [2]:
# variaveis para teste do modelo
cols = ['x1', 'x2', 'x3', 'x4', 'target']
val = [[1,0,1,1, 'A'], [1,1,0,0, 'A'], [1,0,0,1, 'I'], [0,1,1,1, 'I'], [0,1,0,1, 'I']]

teste = pd.DataFrame(val, columns=cols)

In [3]:
teste

,x1,x2,x3,x4,target
0,1,0,1,1,A
1,1,1,0,0,A
2,1,0,0,1,I
3,0,1,1,1,I
4,0,1,0,1,I


###### fitness para classificacao binaria
c1 = previsao_correta_c1/total_c1
c2 = previsao_correta_c2/total_c2

fc = c1*c2

###### fitness para classificacao multi classe
c1 = previsao_correta_c1/total_c1
c2 = previsao_correta_c2/total_c2
c3 = previsao_correta_c3/total_c3

fc = c1 + c2 + c3 ...

###### quatérnio de Holland
Q = b0 + b1x1 + b2x2 + b3x3

###### decaenaquarternio
$\sum[ba1 * xa1 + ba2 * xa2]$

###### hurst para valor aleatorio
$b = -1 + 2 * random$

In [4]:
# separa a variavel preditora dos atributos
x = teste.drop(columns = 'target')
y = np.where(teste.target == 'I', 0, 1)

# cria coluna para calcular b0
b0 = np.ones((x.shape[0],1))
x = x.to_numpy()

x = np.append(b0, x, axis=1)

In [64]:
def gera_aleatorio(num_atibutos):
    return (2 * np.random.rand(num_atibutos)) - 1

def gera_cromossomos(x, b):
    return np.sum(x * b, axis=1)

def fit_cromossomos(x, y, b):
   
    previsao = np.where( gera_cromossomos(x,b) < 0, 0, 1)

    total_corretos = pd.crosstab(y, previsao, dropna=False).reindex(columns=[0,1], fill_value=0)
    total_valores = dict(collections.Counter(y))
        
    return (total_corretos.loc[0,0] * total_corretos.loc[1,1]) / (total_valores[1] * total_valores[0])

def seleciona_pais(fc_df):
    escolhidos = np.random.rand(2)
    fc_df['total'] = fc_df['fc_cromossomo'].sum()
    fc_df['proporcao'] = fc_df['fc_cromossomo'] / fc_df['total']
    fc_df = fc_df.fillna(0)
    
    if(any(fc_df['proporcao'] == 1) | all(fc_df['proporcao'] == 0)):
        prop = None
    else:
        prop = fc_df['proporcao']
        
    return np.random.choice(fc_df.index, size=2, p=prop, replace=False)

def crossover(fc, bc, escolhidos, x, y):
    fc_filho = []
    bc_filho = []
    
    tamanho = x.shape[1]
    cross = np.random.randint(0,tamanho,3)
    
    fc_menor_indice = fc.nsmallest(2, 'fc_cromossomo').index

    for parte in cross:
        cromo_subistitui = np.random.randint(0,tamanho,2)
        cromo_ind = np.random.randint(0,6)
        
        filho = np.concatenate( (bc[escolhidos[0]][:parte+1], bc[escolhidos[1]][parte+1:]) )
        filho[cromo_subistitui[0]] = bc[cromo_ind][cromo_subistitui[1]]
        
        fc_filho.append(fit_cromossomos(x,y,filho))
        bc_filho.append(filho)
        
    bc_filho = dict(enumerate(bc_filho))
    fc_filho = pd.DataFrame.from_dict(dict(enumerate(fc_filho)), orient = 'index', columns = ['fc_cromossomo'])
    fc_filho = fc_filho.iloc[fc_filho.nlargest(2, 'fc_cromossomo').index]
    
    for i, menor in  enumerate(fc_menor_indice):
        bc[menor] = bc_filho[i]
        fc.iloc[menor] = fc_filho.iloc[i]
    
def fit(x, y, num_cromossomos = 6, iteracao = 500):
    
    for i in range(iteracao):
        banco_cromossomos = []
        banco_fit = []
    
        for i in range(num_cromossomos):
            b = gera_aleatorio(x.shape[1])
        
            fc = fit_cromossomos(x,y,b)
        
            banco_cromossomos.append(b)
            banco_fit.append(fc)
        
        banco_cromossomos = dict(enumerate(banco_cromossomos))
        banco_fit = pd.DataFrame.from_dict(dict(enumerate(banco_fit)), orient = 'index', columns = ['fc_cromossomo'])
    
        escolhidos = seleciona_pais(banco_fit.copy())
        crossover(banco_fit, banco_cromossomos, escolhidos, x, y)

    return banco_cromossomos[banco_fit.nlargest(1, 'fc_cromossomo').index[0]]

In [66]:
fit(x,y)

array([-0.42335041, -0.34511003,  0.93025511, -0.27307852, -0.81757346])

In [ ]:
#testes
'''# convert numerico para categorical
cols = df.drop(columns='loan_status').select_dtypes(include=['float64','int64']).columns

df.person_age = pd.cut(df.person_age, bins=[19,30,40,50,150], labels=['ate 30', 'ate 40', 'ate 50', 'acima 50'])
df.person_income = pd.cut(df.person_income, bins=4, labels=['Faixa salario 1', 'Faixa salario 2', 
                                                            'Faixa salario 3', 'Faixa salario 4'])
df.person_emp_length = pd.cut(df.person_emp_length, bins=[-1,2,5,10,20,30,40,50], 
                              labels=['Faixa emprego 1', 'Faixa emprego 2', 'Faixa emprego 3', 
                                      'Faixa emrpego 4', 'Faixa emprego 5', 'Faixa emprego 6',
                                      'Faixa emprego 7'])
df.loan_amnt = pd.cut(df.loan_amnt, bins=4, labels=['Faixa loan 1', 'Faixa loan 2', 'Faixa loan 3', 'Faixa loan 4'])
df.loan_int_rate = pd.cut(df.loan_int_rate, bins=5, labels=['Faixa rate 1', 'Faixa rate 2', 'Faixa rate 3', 
                                                            'Faixa rate 4', 'Faixa rate 5'])
df.loan_percent_income = pd.cut(df.loan_percent_income, bins=5, labels=['Faixa perc 1', 'Faixa perc 2', 'Faixa perc 3', 
                                                                        'Faixa perc 4', 'Faixa perc 5'])
df.cb_person_cred_hist_length = pd.cut(df.cb_person_cred_hist_length, bins=5, 
                                       labels=['Faixa cred 1', 'Faixa cred 2', 'Faixa cred 3', 
                                               'Faixa cred 4', 'Faixa cred 5'])'''